In [ ]:
import os
import numpy as np
import Cell_BLAST as cb
import utils
os.environ["CUDA_VISIBLE_DEVICES"] = utils.pick_gpu_lowest_memory()
cb.config.RANDOM_SEED = 0
cb.config.N_JOBS = 4
fixed_model_kwargs = dict(
    latent_dim=10, cat_dim=20,
    epoch=500, patience=20
)

In [ ]:
cb.__version__

---

# Farrell

In [ ]:
farrell = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Farrell/data.h5")
utils.peek(farrell, "build/zebrafish_embryo/Farrell")
farrell.obs.head()

In [ ]:
farrell.obs.dtypes

In [ ]:
farrell_model = cb.directi.fit_DIRECTi(
    farrell, farrell.uns["seurat_genes"],
    **fixed_model_kwargs
)
farrell.latent = farrell_model.inference(farrell)

In [ ]:
farrell.obs["probabilistic_breadth"] = farrell.obs["probabilistic_breadth"].astype(np.float)

In [ ]:
ax = farrell.visualize_latent("probabilistic_breadth", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/zebrafish_embryo/Farrell/probabilistic_breadth.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
farrell.write_dataset("build/zebrafish_embryo/Farrell/Farrell.h5")

In [ ]:
%%capture capio
farrell_models = [farrell_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    farrell_models.append(cb.directi.fit_DIRECTi(
        farrell, farrell.uns["seurat_genes"],
        **fixed_model_kwargs, 
        random_seed=i
    ))
farrell_blast = cb.blast.BLAST(farrell_models, farrell)
farrell_blast.save("build/zebrafish_embryo/Farrell")

In [ ]:
with open("build/zebrafish_embryo/Farrell/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/zebrafish_embryo/Farrell/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
%%writefile "build/zebrafish_embryo/Farrell/predictable.txt"
probabilistic_breadth

---

# Wagner

In [ ]:
wagner = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Wagner/data.h5")
utils.peek(wagner, "build/zebrafish_embryo/Wagner")
wagner.obs.head()

In [ ]:
wagner.obs.dtypes

In [ ]:
wagner_model = cb.directi.fit_DIRECTi(
    wagner, wagner.uns["seurat_genes"],
    **fixed_model_kwargs
)
wagner.latent = wagner_model.inference(wagner)

In [ ]:
ax = wagner.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/zebrafish_embryo/Wagner/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = wagner.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/zebrafish_embryo/Wagner/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = wagner.visualize_latent("lifestage", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/zebrafish_embryo/Wagner/lifestage.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
wagner.write_dataset("build/zebrafish_embryo/Wagner/Wagner.h5")

In [ ]:
%%capture capio
wagner_models = [wagner_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    wagner_models.append(cb.directi.fit_DIRECTi(
        wagner, wagner.uns["seurat_genes"],
        **fixed_model_kwargs, 
        random_seed=i
    ))
wagner_blast = cb.blast.BLAST(wagner_models, wagner)
wagner_blast.save("build/zebrafish_embryo/Wagner")

In [ ]:
with open("build/zebrafish_embryo/Wagner/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/zebrafish_embryo/Wagner/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(wagner_blast, "build/zebrafish_embryo/Wagner")

In [ ]:
%%writefile "build/zebrafish_embryo/Wagner/predictable.txt"
cell_ontology_class
cell_type1
lifestage